# SQL: Permissões

Desde nossas primeiras aulas, realizamos conexões com o **SGBD** (Sistema de Gerenciamento de Bancos de Dados) MySQL, seja no Workbench ou pelos jupyter notebooks.

Abra o arquivo `.env` que utilizamos para configurar as variáveis de ambiente. Provavelmente você deve encontrar uma linha com este conteúdo:

`MD_DB_USERNAME="root"`

Bem, isto significa que estamos nos conectando utilizando o usuário `root`!

**<div id="user_root"></div>**
**O que é o usuário root no MySQL?**

O usuário `root` é um **superusuário** no MySQL. Ele tem acesso total ao SGBD, funcionando como um usuário administrador. Possui privilégios ilimitados para criar, modificar e excluir bancos de dados, bem como conceder ou revogar permissões a outros usuários. 

Ele é criado durante a instalação do MySQL e é altamente recomendado que sejam tomadas medidas de segurança para proteger essa conta, como definir uma senha forte e restringir o acesso remoto apenas aos endereços IP autorizados.

## E a aplicação em produção?!

Estamos desenhando soluções de banco de dados que um dia serão entregues a algum cliente (estarão em produção, prontas para serem integradas a outros sistemas). Quando a conexão com o SGBD for exposta para uso por alguma API (como a que desenvolveram no projeto), vamos precisar de um usuário e senha para conexão com o MySQL.

A resposta em <a href="#user_root">"O que é o usuário root no MySQL?"</a> já dá uma ideia que utilizar o usuário `root` nestas situações não parece correto. Caso ocorra algum vazamento da senha ou *SQL injection*, a base poderia ficar exposta tanto para leitura quanto para escrita, ou seja, um hacker poderia tanto **ver os dados** nas tabelas quanto **editar** e **exluir**.

## Qual a alternativa?!

Uma alternativa mais adequada envolve:

- **Criar usuários únicos**: vamos criar múltiplos usuário com acesso ao banco. Cada indivíduo ou aplicação em *deploy* terá seu usuário e senha personalizado (sem compartilhamento de senhas - Netflix vibes!)


- **Dar permissões**: vamos adicionar ao usuário apenas as permissões necessárias para que seu papel seja cumprido. Por exemplo, um colaborador de uma consultoria externa que presta serviços à uma empresa precisa ter acesso à base toda? Não! Ele provavelmente precisará apenas ler dados e não escrever, além de ser indicado que tenha permissão de visualização apenas em parte das tabelas (as que forem úteis para desenvolvimento do projeto).

Antes de praticar, vamos importas as bibliotecas necessárias

In [1]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

load_dotenv()

def run_query(connection, query, args=None):
    """
    Função que recebe uma conexão, query e argumentos;
    executa a query na conexão, e retorna os resultados
    """
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)

def get_connection_helper():

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database="fiscamuni",
    )
    return connection, partial(run_query, connection)

## Criar usuário no MySQL

Podemos criar usuário no MySQL com a seguinte sintaxe:

```MySQL
CREATE USER '<user>'@'<host>' IDENTIFIED BY '<password>';
```

Vamos experimentar! Abra o Workbench e execute:

```MySQL
CREATE USER 'joao'@'localhost' IDENTIFIED BY 'abc123';
```

Então, vamos tentar fazer uma conexão com o novo usuário. A dinâmica da aula será utilizar o Workbench com o usuário `root` e os notebooks com os usuário criados para testar as permissões.

<div class="alert alert-warning">

Provavelmente você verá em alguns materiais a indicação do uso do comando
```SQL
FLUSH PRIVILEGES;
```

<br>
após alterar permissões de um usuário. Ele força a atualização das permissões pelo servidor. Os `GRANTS` tomam efeito assim que executados, mas o uso do `FLUSH` pode ser necessário em alguns casos.
</div>

In [2]:
conn1 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="musica"
)

ProgrammingError: 1044 (42000): Access denied for user 'joao'@'localhost' to database 'musica'

Perceba que a conexão falhou! Isto porque, apesar do usuário ter sido criado, ele não possui as permissões necessárias!

## Dar Permissão

Para alterar permissões de um usuário, iremos utilizar `GRANT`.

A seguinte sintaxe resumida pode ser utilizada:

```mysql
GRANT PRIVILEGE ON database.table TO 'username'@'host';
```

Veja mais em https://dev.mysql.com/doc/refman/8.0/en/grant.html

Abra o Workbench e execute:

```mysql
GRANT SELECT ON musica.* TO 'joao'@'localhost';
```

Assim, o usuário terá a permissão de **SELECT** em todas as tabelas do database `musica`. Vamos testar!

In [4]:
conn1 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="musica"
)

In [5]:
run_query(conn1, "SELECT * FROM AUTOR")

Executando query:
(1, 'Renato Russo')
(2, 'Tom Jobim')
(3, 'Chico Buarque')
(4, 'Dado Villa-Lobos')
(5, 'Marcelo Bonfá')
(6, 'Ico Ouro-Preto')
(7, 'Vinicius de Moraes')
(8, 'Baden Powell')
(9, 'Paulo Cesar Pinheiro')
(10, 'João Bosco')
(11, 'Aldir Blanc')
(12, 'Joyce')
(13, 'Ana Terra')
(14, 'Cartola')
(15, 'Cláudio Tolomei')
(16, 'João Nogueira')
(17, 'Suely Costa')
(18, 'Guinga')
(19, 'Danilo Caymmi')
(20, 'Tunai')
(21, 'Sérgio Natureza')
(22, 'Heitor Villa Lobos')
(23, 'Ferreira Gullar')
(24, 'Catulo da Paixão Cearense')
(25, 'Zezé di Camargo')
(26, 'Niltinho Edilberto')
(27, 'Marisa Monte')
(28, 'Carlinhos Brown')
(29, 'Gonzaga Jr')
(30, 'Roberto Mendes')
(31, 'Ana Basbaum')
(32, 'Caetano Veloso')
(33, 'José Miguel Wisnik')
(34, 'Vevé Calazans')
(35, 'Gerônimo')
(36, 'Sérgio Natureza')
(37, 'Roberto Carlos')
(38, 'Erasmo Carlos')
(39, 'Renato Teixeira')
(40, 'Chico César')
(41, 'Vanessa da Mata')
(42, 'Jorge Portugal')
(43, 'Lilian Knapp')
(44, 'Renato Barros')
(45, 'Bebel Gilberto

In [6]:
run_query(conn1, "SELECT * FROM CD")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


Pronto! Agora a conexão está funcionando! Vamos fechar a conexão...

In [7]:
# Não vamos manter a conexão aberta!
conn1.close()

... e tentar realizar a conexão com a base de dados `tranqueira`:

In [8]:
conn2 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="tranqueira"
)

ProgrammingError: 1044 (42000): Access denied for user 'joao'@'localhost' to database 'tranqueira'

Obtivemos novamente um erro. Caso o usuário `joao` realmente necessite acesso à base `tranqueira`, um novo `GRANT` deve ser realizado!

Ao fazer *deploy* de aplicações em produção, é indicado ter usuários diferentes por aplicação, apenas as permissões necessárias.

Vamos criar um segundo usuário com permissão de `SELECT` na base `tranqueira`:
    
```mysql
CREATE USER 'ana'@'localhost' IDENTIFIED BY '456456';
GRANT SELECT ON tranqueira.* TO 'ana'@'localhost';
```
Teste a conexão:

In [9]:
conn3 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

Vamos ver se conseguimos ler a tabela `perigo`:

In [10]:
run_query(conn3, "SELECT * FROM perigo")

Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Mental')


E fazer um **INSERT**, também na tabela `perigo`:

In [13]:
sql = """
INSERT INTO `tranqueira`.`perigo`
    (`id`,`Nome`)
VALUES
    (10,'Moral')
"""

run_query(conn3, sql);

Executando query:


Novamente, precisamos dar permissão de inserção! Dê a permissão pelo Workbench e teste novamente! Perceba que agora a permissão será para **apenas uma tabela**:

```mysql
GRANT INSERT ON tranqueira.perigo TO 'ana'@'localhost';
```

In [14]:
conn3.rollback()

In [15]:
sql = """
INSERT INTO `tranqueira`.`perigo`
    (`id`, `Nome`)
VALUES
    (10,'Moral')
"""

run_query(conn3, sql);

Executando query:


Vamos verificar se realmente conseguimos inserir:

In [16]:
run_query(conn3, "SELECT * FROM perigo")

Executando query:
(1, 'Cardiaco')
(2, 'Intestinal')
(3, 'Dermatologico')
(4, 'Mental')
(10, 'Moral')


Então, desfazemos a inserção e fechamos a conexão!

In [19]:
conn3.rollback()
conn3.close()

OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 10038 An operation was attempted on something that is not a socket

<div class="alert alert-warning">

Algums exemplos de GRANTS!
    
```mysql
GRANT ALL PRIVILEGES ON *.* TO 'maria'@'localhost' WITH GRANT OPTION;
GRANT CREATE TEMPORARY TABLES ON coemu.* TO 'user_deploy_api'@'localhost';
GRANT EXECUTE ON sys.* TO 'user_dashboard'@'localhost' WITH GRANT OPTION;
GRANT SELECT, SHOW VIEW ON cartracking.* TO 'leitor'@'localhost';
```
    
após alterar permissões de um usuário. Ele força a atualização das permissões pelo servidor, mas geralmente é desnecessário pois os `GRANTS`tomam efeito assim que executados.
</div>

## Revogar Permissão

Para ver as permissões de um usuário, utilize

```mysql
SHOW GRANTS FOR 'ana'@'localhost';
```


Caso queira revogar permissões de um usuário, iremos utilizar `REVOKE`.

```mysql
REVOKE SELECT ON tranqueira.* FROM 'ana'@'localhost';
```

In [21]:
conn4 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

Conseguimos fazer o login. Vamos testar se o **SELECT** funciona!

In [22]:
run_query(conn4, "SELECT * FROM perigo;")

Executando query:


ProgrammingError: 1142 (42000): SELECT command denied to user 'ana'@'localhost' for table 'perigo'

Vamos analisar os grants restantes ao usuário:

In [23]:
run_query(conn4, "SHOW GRANTS FOR 'ana'@'localhost';")

Executando query:
('GRANT USAGE ON *.* TO `ana`@`localhost`',)
('GRANT INSERT ON `tranqueira`.* TO `ana`@`localhost`',)
('GRANT INSERT ON `tranqueira`.`perigo` TO `ana`@`localhost`',)


Temos o `GRANT INSERT` mas não mais o `GRANT SELECT`, então nossa única permissão é para inserir linhas na tabela `perigo`.

In [ ]:
conn4.close()

Remova também esta permissão:

```mysql
REVOKE INSERT ON tranqueira.perigo FROM 'ana'@'localhost';
```

E teste novamente:

In [ ]:
conn5 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

## Hosts

Ao criar um usuário no MySQL, podemos especificar de qual **host** ele pode se conectar. Nos exemplos que apresentamos, o usuário `joao` só poderá se conectar ao MySQL do `localhost`.

Para permitir que o usuário se conecte de outros hosts, devemos criar um novo usuário com o mesmo nome e senha, mas com uma configuração de `host` diferente.

Por exemplo, para permitir que o usuário `joao` se conecte de qualquer host, você pode criar um novo usuário com a seguinte instrução SQL:

```mysql
CREATE USER 'joao'@'%' IDENTIFIED BY 'abc123';
```

No exemplo acima, o símbolo `%` indica que o usuário pode se conectar de qualquer host.

Então, basta condeder as permissões adequadas. Se quiser todas as permissões para o usuário "joao" ao banco de dados `tranqueira`, execute a seguinte instrução SQL:

```mysql
GRANT ALL PRIVILEGES ON tranqueira.* TO 'joao'@'%';
```

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [ ]:
ia.tasks()

In [ ]:
ia.grades(task="permissions")

Vamos criar nossa tradicional conexão!

In [ ]:
root_connection, db = get_connection_helper()

## Base de dados

Utilizaremos a base de dados `fiscamuni`, que busca armazenar informações sobre fiscais e multas aplicadas à propriedades pertecentes a uma empresa.

A base possui o seguinte modelo relacional:

<img src="img/eer_diagram.png">

Execute o script `multas.sql` no Workbench para gerar a base.

**Exercício 1**:

Crie um usuário `camilaw2` com login a partir do `localhost` e senha `699a1deacb58`.

In [ ]:
sql_ex01 = """
-- SUA QUERY AQUI!
"""

db(sql_ex01)

In [ ]:
ia.sender(answer="sql_ex01", task="permissions", question="ex01", answer_type="pyvar")

**Exercício 2**:

Remova o usuário criado no exercício anterior.

In [ ]:
sql_ex02 = """
-- SUA QUERY AQUI!
"""

db(sql_ex02)

In [ ]:
ia.sender(answer="sql_ex02", task="permissions", question="ex02", answer_type="pyvar")

**Exercício 3**:

Crie um usuário `marianafag` com login a partir de **qualquer host** e senha `cB18cDd2503F`.

In [ ]:
sql_ex03 = """
-- SUA QUERY AQUI!
"""

db(sql_ex03)

In [ ]:
ia.sender(answer="sql_ex03", task="permissions", question="ex03", answer_type="pyvar")

**Exercício 4**:

Crie um usuário `pereiradjs` com login a partir do IP `192.168.15.160` e senha `bb3_091#2d6@A70`.

In [ ]:
sql_ex04 = """
-- SUA QUERY AQUI!
"""

db(sql_ex04)

In [ ]:
ia.sender(answer="sql_ex04", task="permissions", question="ex04", answer_type="pyvar")

**Exercício 5**:

Utilize SQL para criar um usuário, observando que:

- O nome de usuário será `diniz`
- A senha será `abc123cba`
- Deve conseguir login de qualquer host

Ainda, o usuário deve ter permissões apenas de:

- **Leitura** na tabela `fiscal`. 

Ou seja, inserção e deleção ou select em outras tabelas ou bases de dados devem estar bloqueadas.

In [ ]:
sql_ex05 = """
-- SUA QUERY AQUI!
"""

db(sql_ex05)

In [ ]:
ia.sender(answer="sql_ex05", task="permissions", question="ex05", answer_type="pyvar")

**Exercício 6**:

Crie um usuário `rem_dash_alu` com login a partir de IPs da **subnet /24** em `192.168.58.0` com senha `9C26189563A7`.

In [ ]:
sql_ex06 = """
-- SUA QUERY AQUI!
"""

db(sql_ex06)

In [ ]:
ia.sender(answer="sql_ex06", task="permissions", question="ex06", answer_type="pyvar")

**Exercício 7**:

Suponha que um exista um usuário `marianatt` com permissão de login de **qualquer host**.

Altere a senha do usuário para `b2a8b85f76b1b923`.

In [ ]:
sql_ex07 = """
-- SUA QUERY AQUI!
"""

db(sql_ex07)

In [ ]:
ia.sender(answer="sql_ex07", task="permissions", question="ex07", answer_type="pyvar")

**Exercício 8**:

Escreva uma query que liste:

- O `id_empresa` da empresa.
- A quantidade de propriedades que a empresa possui cadastrada na base. Aqui, a coluna deve se chamar `qtde_propriedades`.
- O valor total de multas da empresa. Aqui, a coluna deve se chamar `valor_total_multas`.

**Obs**:
- Empresas sem propriedades devem ser retornadas com valor zerado (`0`) em `qtde_propriedades`.
- Empresas sem multas devem ser retornadas com valor zerado (`0.00`) em `valor_total_multas`.

Exiba ordenado por:
- `valor_total_multas` (decrescente) e `qtde_propriedades` (decrescente)

In [ ]:
sql_ex08 = """
-- SUA QUERY AQUI!
"""

db(sql_ex08)

In [ ]:
ia.sender(answer="sql_ex08", task="permissions", question="ex08", answer_type="pyvar")

**Exercício 9**:

Transforme a query do exercício anterior em uma **view** chamada `abt_empresa_total`.

Envie apenas o comando de criação da view, não envie `DROP`!

In [ ]:
sql_ex09 = """
-- SUA QUERY AQUI!
"""

db(sql_ex09)

In [ ]:
ia.sender(answer="sql_ex09", task="permissions", question="ex09", answer_type="pyvar")

**Exercício 10**:

Sua empresa contratou uma consultoria para atuar em um projeto.

Relações entre empresas sempre expoem problemas de confiança, onde uma não quer que a outra tenha acesso a todos os seus dados.

Assim:
- Crie um usuário `caiomc_consult` com permissão de login de **qualquer host** e senha `6b7997f42e0ebf3a51d2`.
- O consultor deve ter permissão **apenas** de **LEITURA** na **view** `abt_empresa_total`.

Envia todas as queries em uma única string, separadas por `;`

In [ ]:
sql_ex10 = """
-- SUA QUERY AQUI!
"""

db(sql_ex10)

In [ ]:
ia.sender(answer="sql_ex10", task="permissions", question="ex10", answer_type="pyvar")

**Exercício 11**:

Foi criada uma aplicação que captura dados e faz a ingestão deles na base.

A política aceita pela empresa é de que a aplicação deve ter as permissões de:

- **INSERIR** e **LER** das tabelas da base `fiscamuni`:
    - empresa
    - fiscal
    - multa
    - propriedade
- **LER** das tabelas e views da base `fiscamuni`:
    - abt_empresa_total
    - motivo

Considere que a aplicação utiliza um usuário `u_ingest_multa` já existente com permissão de login de **qualquer host** e senha `e7854285319f1c83fcd1`.

Envia todas as queries em uma única string, separadas por `;`

In [ ]:
sql_ex11 = """
-- SUA QUERY AQUI!
"""

db(sql_ex11)

In [ ]:
ia.sender(answer="sql_ex11", task="permissions", question="ex11", answer_type="pyvar")

**Exercício 12**:

Considere que o usuário `u_ingest_multa` tem todas as permissões relatadas no exercício anterior.

Ele não irá mais necessitar **INSERIR** na tabela `empresa`, nem **LER**/**INSERIR** na tabela `fiscal`.

Faça as atualizações, revogando as permissões não mais necessárias.

In [ ]:
sql_ex12 = """
-- SUA QUERY AQUI!
"""

db(sql_ex12)

In [ ]:
ia.sender(answer="sql_ex12", task="permissions", question="ex12", answer_type="pyvar")

**Exercício 13**:

O consultor (o mesmo dos exercícios anteriores) necessita de acesso a mais dados.

Porém, a empresa não quer liberar o acesso completo à base (contento o nome dos fiscais, nome das propriedades multadas e demais informações sensíveis).

**a)** Que solução você utilizaria neste caso?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** Vamos supor que retornar os IDs não seja suficiente. Isto ocorre, por exemplo, quando você quer retornar o endereço de alguém, permitindo que o analista identifique que são endereços diferentes, mas sem saber exatamente qual rua.

Note que podemos ter muitas pessoas com o mesmo endereço(ex: mesma rua), e todas devem estar com o mesmo valor no campo.

Para este caso, uma sugestão é aplicar uma função de HASH, como SHA256.

Crie uma **view** `propriedade_consult` que contenha o `id` e o SHA256 da `descricao`, `cidade` e `endereco` da tabela `propriedade`. Mantenha o nome original das colunas.

Assim, o usuário utilizado pela consultoria poderia ter permissão de leitura apenas na **view** `propriedade_consult` e não na tabela original. Esta parte não precisa fazer, se quiser, teste localmente! 

In [ ]:
sql_ex13b = """
-- SUA QUERY AQUI!
"""

db(sql_ex13b)

In [ ]:
ia.sender(answer="sql_ex13b", task="permissions", question="ex13b", answer_type="pyvar")

**Exercício 14**:

Considere o retorno de uma coluna de **CPF** uma tabela utilizando SHA256.

Por exemplo:
```mysql
SELECT SHA2('377.662.560-02', 256)
UNION
SELECT SHA2('404.483.920-46', 256)
UNION
SELECT SHA2('196.499.400-49', 256)
UNION
SELECT SHA2('895.322.380-69', 256);
```

Analise a seguinte afirmação: Tendo os hashs e sabendo que é um campo de CPF, é impossível descobrir os CPFs originais.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 15**:

Pesquise sobre anonimização de dados e mascaramento de dados. Explique a importância e como funciona.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 16**:

Esqueça o usuário do banco de dados nesta questão, uma vez que ele é o usuário utilizado pelas aplicações em deploy e engenheiros da empresa.

Suponha que você foi contratado para criar uma aplicação que necessita de **login**. Os usuário devem possuir, pelo menos os campos de `id`, `nickname` e `senha`.

**a)** Construa a DDL de criação da tabela de `usuario`.

<div class="alert alert-success">

```mysql
-- SUA DDL aqui!
```
</div>

**b)** Qual seria a query para realizar uma inserção de um usuário nesta tabela? Utilize dados fictícios!

<div class="alert alert-success">

```mysql
-- SUA DML aqui!
```
</div>

**c)** Como você armazenou a senha no exercício "b)"? Você deixou como *plain-text*? Se sim, explique se foi uma boa escolha e quais as consequências!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**d)** Quais seriam alternativas melhores para armanenar dados sensíveis (como senhas) em banco de dados?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**e)** Pesquise sobre *Salting & peppering passwords*. Explique como funciona!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**f)** Pesquise sobre *senhas e entropia*. Anote abaixo os principais aprendizados!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

### Fechando a conexão

In [ ]:
root_connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

In [ ]:
ia.grades(by="task", task="permissions")

In [ ]:
ia.grades(task="permissions")

In [ ]:
ia.grades(by="task")

Até a próxima aula!